In [1]:
from PIL import Image
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
# import tensorflowjs as tfjs

print(f"The Tensorflow version is {tf.__version__}")
print(f"The Keras version is {tf.keras.__version__}")

The Tensorflow version is 1.15.0
The Keras version is 2.2.4-tf


In [2]:
IMG_SIZE = (224,224)
np.shape(plt.imread("./Data/train/NORMAL/IM-0115-0001.jpeg"))

(1858, 2090)

In [3]:
train_generator = ImageDataGenerator(shear_range=10
                                   ,zoom_range=0.2
                                   ,horizontal_flip=True
                                   ,preprocessing_function=preprocess_input)

valid_generator = ImageDataGenerator(preprocessing_function=preprocess_input)

In [4]:
train_data = train_generator.flow_from_directory("./Data/train/",target_size=IMG_SIZE,color_mode="rgb",batch_size=32)
valid_data = valid_generator.flow_from_directory("./Data/val/",target_size=IMG_SIZE,color_mode="rgb",batch_size=32)

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


In [5]:
train_data.class_indices

{'NORMAL': 0, 'PNEUMONIA': 1}

In [6]:
base_model = VGG16(include_top=False,weights="imagenet",input_shape=(224,224,3))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [7]:
for layers in base_model.layers:
    print(layers)
    layers.trainable = False
base_model.input

<tf.Tensor 'input_1:0' shape=(?, 224, 224, 3) dtype=float32>

In [8]:
from tensorflow.keras import Model, layers
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(128,activation='relu')(x)
x = layers.Dense(128,activation='relu')(x)
predictions = layers.Dense(2, activation='softmax')(x)
model = Model(base_model.input, predictions)

In [9]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [10]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [11]:
history = model.fit(train_data,steps_per_epoch=10,epochs=5,validation_data=valid_data)

Epoch 1/5
10/10 [==============================] - 19s 2s/step - loss: 1.1651 - acc: 0.6594 - val_loss: 0.6283 - val_acc: 0.6875
Epoch 2/5
10/10 [==============================] - 9s 891ms/step - loss: 0.2804 - acc: 0.8813 - val_loss: 0.4715 - val_acc: 0.8125
Epoch 3/5
10/10 [==============================] - 9s 902ms/step - loss: 0.2247 - acc: 0.9062 - val_loss: 0.9172 - val_acc: 0.6875
Epoch 4/5
10/10 [==============================] - 9s 893ms/step - loss: 0.2745 - acc: 0.9094 - val_loss: 0.4443 - val_acc: 0.8125
Epoch 5/5
10/10 [==============================] - 9s 939ms/step - loss: 0.2326 - acc: 0.9062 - val_loss: 0.3322 - val_acc: 0.8125


In [12]:
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(model,'model_for_web',weight_shard_size_bytes=int(1e9))

C:\Users\CHAKARAPANI\AppData\Roaming\Python\Python36\site-packages\tensorflowjs\converters\keras_h5_conversion.py:123: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  return h5py.File(h5file)
